In [ ]:
# 迭代器模式 Iterator pattern 一种惰性获取数据项的方法，按需一次获取一个数据项
# yield关键字 用于构建生成器 generator，作用域迭代器一样。
# 所有的生成器都是迭代器（完全实现了迭代器的接口），不过迭代器用于从集合中取出元素，而生成器用于“凭空”生成元素。

# 在Python中所有集合都可以迭代。 迭代器用于支持：
# - for循环
# - 构建和扩展集合类型
# - 逐行遍历文本文件
# - 列表推导、字典推导和集合推导
# - 元组拆包
# - 调用函数时，使用*拆包实参


# 序列可以迭代，会自动调用iter函数， 内置的iter函数：
# - 检查对象是否实现了__iter__方法，如果有则调用，获得一个迭代器
# - 如果没有，但是实现了__getitem__方法，则解释器会创建一个迭代器，尝试按顺序（从索引0开始）获取元素
# - 如果尝试失败，则抛出TypeError异常，提示“C object is not iterable” 其中C是母队对象所属的类。

import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)  # 利用正则表表达式得到全部单词（以空格分隔）列表

    def __getitem__(self, index):
        return self.words[index]  # 实现可迭代对象接口

    def __len__(self, index): 
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)  # 长字符串的简略显示形式

s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)

In [ ]:
# abc.Iterable类实现了__subclasshook__方法，通过检查是否实现了__iter__方法来判定对象是否可迭代。

class Foo:
    def __iter__(self):
        pass

from collections import abc

print(issubclass(Foo, abc.Iterable))
print(issubclass(Sentence, abc.Iterable)) # Sentence 可迭代，但未实现__iter__方法，所以不是Iterable的子类

# 通常不用isinstance(x, abc.Iterable)来判断x是否可迭代（没有考虑实现__getitem__方法来迭代的情况），而是用try捕获TypeError异常

# 构造迭代器的过程
it = iter(s)
for i in range(7):
    print(next(it))

try:
    next(it)
except StopIteration as e:
    print('--stop iter--')

print(list(it))
print(list(iter(s)))


In [ ]:
# 使用生成器函数完成迭代
class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self): # 可以实现为 return iter(self.words)
        for word in self.words:  # 迭代self.words 
            yield word  # 产出当前的word
        return  # 可以不用。

s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)

# 函数定义体中有yield关键字，该函数就是生成器函数。调用生成器函数时，会返回一个生成器对象。生成器函数时生成器工厂。

def gen_123():
    print('gen--->1')
    yield 1
    print('gen--->2')
    yield 2
    print('gen--->3')
    yield 3
    print('gen---end')

print(gen_123())
for i in gen_123():
    print('<<<', i)
print('-----------------')
g = gen_123()
print(next(g))
print(next(g))
print(next(g))
print('-----------------')
res1 = [x*3 for x in gen_123()] # 列表博士生成器函数的生成器对象产出元素
for i in res1:
    print('<<<', i)

print('-----------------')
res2 = (x*3 for x in gen_123()) # res2是一个生成器对象，但生成器未被使用。惰性求值 lazy evaluation
for i in res2:
    print('>>>', i)



In [ ]:
# 使用生成器表达式
class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    # match.group() 提取匹配正则表达式的具体文本
    # re.finditer函数是re.findall函数的惰性版本，返回的不是列表，而是一个生成器，按需生成
    # re.MatchObject实例。如果有很多匹配，惰性版本能节省大量内存。
    # 使用生成器表达式不使用yield
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

s = Sentence('"The time has come," the Walrus said,')
for word in s:
    print(word)


In [ ]:
# 等差数列示例 - 可迭代对象
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):  # <1>
        self.begin = begin
        self.step = step
        self.end = end  # None -> "infinite" series

    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)  # <2>
        forever = self.end is None  # <3>
        index = 0
        while forever or result < self.end:  # <4>
            yield result  # <5>
            index += 1
            result = self.begin + self.step * index  # <6>

ap = ArithmeticProgression(0, 1, 3)
print(list(ap))
ap = ArithmeticProgression(0, 1/3, 1)
print(list(ap))
from fractions import Fraction
from decimal import Decimal
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
print(list(ap))
ap = ArithmeticProgression(0, Decimal('.1'), .3)
print(list(ap))


In [ ]:
# 等差数列 - 生成器函数实现
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

ap = aritprog_gen(0, 1, 3)
print(list(ap))
ap = aritprog_gen(0, 1/3, 1)
print(list(ap))
ap = aritprog_gen(0, Fraction(1, 3), 1)
print(list(ap))
ap = aritprog_gen(0, Decimal('.1'), .3)
print(list(ap)) 


In [ ]:
# 等差数列 - 使用itertools模块

# itertools提供了19个生成器函数

import itertools

# itertools.count(start, step) 函数不会停止，通常结合itertools.takewhile函数一起
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
print(list(gen))

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

ap = aritprog_gen(0, 1, 3)
print(list(ap))
ap = aritprog_gen(0, 1/3, 1)
print(list(ap))
ap = aritprog_gen(0, Fraction(1, 3), 1)
print(list(ap))
ap = aritprog_gen(0, Decimal('.1'), .3)
print(list(ap)) 


In [ ]:
# 当iter函数传入两个参数时，第一个参数必须是常规函数或任何可调用的对象创建迭代器，用于不断调用（无参数）产出各个值；
# 第二个是信号标，产出值为信号标时，触发迭代器抛出StopIteration异常
#
# with open('mydata.txt') as fp:
#    for line in iter(fp.readline, '\n') :
#        process_line(line)
from random import randint

def d6():
    return randint(1, 6) # 取随机整数

for roll in iter(d6, 1):  # 到1停止
    print(roll)

print('------------')
# itertools模块的各个函数
# 可迭代的归约函数：接受一个可迭代对象，返回单个结果：all any max min reduce sum 

def fibonacci():
    a, b = 0 ,1
    while True:
        yield a
        a, b = b, a+b
f = fibonacci()
for i in range(10):
    print(next(f))
